In [ ]:
import requests
import time
import random
import pymongo

client = pymongo.MongoClient(host = "127.0.0.1",port=27017)
db = client["spider"]
collection = db["lagou"]

In [ ]:
def get_cookie():
    headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.80 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
}
    response = requests.get(
        'https://www.lagou.com/jobs/list_?city=%E4%B8%8A%E6%B5%B7&cl=false&fromSearch=true&labelWords=&suginput=',
        headers=headers)  # 请求原网页
    r = requests.utils.dict_from_cookiejar(response.cookies)  # 获取cookies

    cookies = {
        'X_MIDDLE_TOKEN': '797bc148d133274a162ba797a6875817',
        'JSESSIONID': 'ABAAABAAAIAACBI03F33A375F98E05C5108D4D742A34114',
        '_ga': 'GA1.2.1912257997.1548059451',
        '_gat': '1',
        'Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6': '1548059451',
        'user_trace_token': '20190121163050-dbd72da2-1d56-11e9-8927-525400f775ce',
        'LGSID': '20190121163050-dbd72f67-1d56-11e9-8927-525400f775ce',
        'PRE_UTM': '',
        'PRE_HOST': '',
        'PRE_SITE': '',
        'PRE_LAND': 'https%3A%2F%2Fwww.lagou.com%2F%3F_from_mid%3D1',
        'LGUID': '20190121163050-dbd73128-1d56-11e9-8927-525400f775ce',
        '_gid': 'GA1.2.1194828713.1548059451',
        'index_location_city': '%E5%85%A8%E5%9B%BD',
        'TG-TRACK-CODE': 'index_hotjob',
        'LGRID': '20190121163142-fb0cc9c0-1d56-11e9-8928-525400f775ce',
        'Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6': '1548059503',
        'SEARCH_ID': '86ed37f5d8da417dafb53aa25cd6fbc0',
    }
    cookies.update(r)  # 更新接口的cookies
    return cookies

def crawl(city = "", pn = 1, cookies = None):
    headers = {
        'Origin': 'https://www.lagou.com',
        'X-Anit-Forge-Code': '0',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.80 Safari/537.36',
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Referer': 'https://www.lagou.com/jobs/list_java?px=new&city=%E4%B8%8A%E6%B5%B7',
        'X-Requested-With': 'XMLHttpRequest',
        'Connection': 'keep-alive',
        'X-Anit-Forge-Token': 'None',
    }

    params = (
        ('px', 'default'),
        ('city',city),
        ('needAddtionalResult', 'false'),
    )

    data = {"first":"true",
            'kd': '数据分析',
            'pn': pn}
    if pn>1:
        data["first"] = "false"
    response = requests.post('https://www.lagou.com/jobs/positionAjax.json', headers=headers, params=params,
                             cookies=cookies, data=data)  # 请求接口
    return response.json()

In [ ]:
city_list = ["北京","上海","深圳","广州","杭州","成都","南京","武汉","西安","厦门","长沙","苏州","天津"]

for city in city_list:
    print("*"*60)
    print("{city} start".format(city=city))
    for i in range(1,31):
        if (i-1)%5==0:
            cookies = get_cookie()
        time.sleep(random.random()+random.randint(1,2))
        response_json = crawl(city=city,pn=i,cookies=cookies)
        try:
            position_list = response_json["content"][ 'positionResult']["result"]
        except:
            print(response_json)
        if len(position_list)<1: 
            print("{city} start".format(city=city))
            print("*"*60)
            break
        collection.insert_many(position_list)
        print(cookies)
    print("{city} end".format(city=city))
    print("*"*60)